****Import Pandas and Numpy Library****

Veriler üzerinde işem gerçekleştirebilmek için numpy ve pandas kütüphanelerini projeye dahil ediyoruz

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

**Read Csv**

In [ ]:
df=pd.read_csv('../input/bank.csv')

Görselleştirme için gerekli kütüphanelerin dahil edilmesi

In [ ]:
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.cluster import KMeans
from sklearn import datasets
from io import StringIO
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics
%matplotlib inline

İlk 10 veriyi görelim

In [ ]:
df.head(10)

Veriler hakkında genel bilgileri öğrenelim

In [ ]:
df.info()

Veriler hakkında rakamsal bilgiler alalım

In [ ]:
df.describe()

#veri setimizde kategorisel olarak:

job, marital ,education, default, housing ,loan ,contact,month ,poutcome,deposit

verileri bulunmaktadır.

#Rakamsal olarak

age, balance,day,duration,campaign,pdays, previous

verileri bulunmaktadır.

Kategorisel verilerden deposit i ele alalım.

In [ ]:
y=df['deposit']
X=df
y=y.to_frame()
y

Verilerimizi eğitim ve test datası olarak ayıralım ve çıktılarını gösterelim

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4,random_state=101)
print(y.shape)
print(X.shape)
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train.shape)

In [ ]:
#X_train' deki kolonlarımızı görelim
X_train.columns

In [ ]:
#X_train verilerimizin histogram grafiğine bakalım.
sns.pairplot(X_train,diag_kind='hist')

In [ ]:
y.value_counts()

Kategorisel verilerde işlem yapabilmek için rakamsal değerler verelim

In [ ]:
combine_=[y_train,y_test]
depositmapping={'yes':1,'no': 0}
for dt in combine_:
   dt['deposit']=dt['deposit'].map(depositmapping)
y_train

In [ ]:
#yes olanlara 1, no olanlara 0 ataması yapalım
combine=[X_train,X_test]
depositmapping={'yes':1,'no': 0}
for dt in combine:
   dt['deposit']=dt['deposit'].map(depositmapping)

In [ ]:
#x_train' den ilk 5 satırı görüntülüyelim
X_train.head()

In [ ]:
#y_train den ilk 5 satırı görüntülüyelim
y_train.head()

In [ ]:
#Kategorisel verileri analiz edelim

#Job kategorisel verisini ele alalım.

X_train[['job','deposit']].groupby('job',as_index=False).mean().sort_values('deposit',ascending=False)

Öğrencilerin, emekli ve çalışmayanların para yatırma olasılıklarının daha yüksek olduğunu görüyoruz. 1'i öğrencilere, 2'yi emeklilere, 3'ü işsizlere ve 0'ı diğerlerine atıyoruz

In [ ]:
for df in combine:
    df['job']=df['job'].replace(['management','technician','unknown','admin.','housemaid','self-employed','services',
                                'blue-collar','entrepreneur'],'rare',regex=True)
jobmapping={'student':1,'retired':2,'unemployed':3,'rare':0}
for df in combine:
    df['job']=df['job'].map(jobmapping)
    

In [ ]:
#kişinin medeni durumunu analiz edelim

X_train[['marital','deposit']].groupby('marital',as_index=False).mean().sort_values('deposit',ascending=False)

Kişinin para yatırıp yatırmayacağına karar verirken medeni durumunun pek bir önemi olmadığını görüyoruz.
Bu nedenle bu parametreyi ortadan kaldırıyoruz

In [ ]:
#Eğitim durumunu analiz edelim
X_train[['education','deposit']].groupby('education',as_index=False).mean().sort_values('deposit',ascending=False)

** Yukarıdaki verilerden, kişinin ne kadar eğitimli olursa para yatırma olasılığının o kadar yüksek olduğunu söyleyebiliriz.**
Bu nedenle 1'i birincil, 2'yi ikincil, 3'ü üçüncül ve 0'ı bilinmeyene atarız

In [ ]:
educationmapping={'primary':1,'secondary':2,'tertiary':3,'unknown':0}
for df in combine:
    df['education']=df['education'].map(educationmapping)

In [ ]:
#'default' değişken analizi

X_train[['default','deposit']].groupby('default',as_index=False).mean().sort_values('deposit',ascending=False)

Deposit yatıranların daha fazla para yatırdığını görebiliyoruz
Hayır' a 1, Evet e 0 ataması yapıyoruz.

In [ ]:
defaultmapping={'no':1,'yes':0}
for df in combine:
    df['default']=df['default'].map(defaultmapping)

In [ ]:
#Kredi değişkenini analiz edelim

X_train[['loan','deposit']].groupby('loan',as_index=False).mean().sort_values('deposit',ascending=False)

Kişisel kredisi olmayanların daha fazla para yatırdığını görebiliyoruz.
1'i 'hayır'a ve 0'ı' evet'e atıyoruz

In [ ]:
loanmapping={'no':1,'yes':0}
for df in combine:
    df['loan']=df['loan'].map(defaultmapping)

In [ ]:
# 'contact' değişkenini analiz edelim

X_train[['contact','deposit']].groupby('contact',as_index=False).mean().sort_values('deposit',ascending=False)

Müşteriyle iletişim kurma şeklinin, kişinin para yatırıp yatırmadığıyla çok az ilgisi olduğunu görebiliriz, bu nedenle bu değişkeni kaldırabiliriz

In [ ]:
#'month' değişkenini analiz edelim

X_train[['month','deposit']].groupby('month',as_index=False).mean().sort_values('deposit',ascending=False)

Ayları üç kategoriye ayıracağız.
mar.dec.sep.oct'a 2, 'apr, feb, aug, jun' a 1'i, nov, jul, jan, may'a 0'ı atayalım

In [ ]:
for df in combine:
    df['month']=df['month'].replace(['mar','dec','sep','oct'],2,regex=True)
    df['month']=df['month'].replace(['apr','feb','aug','jun'],1,regex=True)
    df['month']=df['month'].replace(['nov','jul','jan','may'],0,regex=True)

In [ ]:
#'poutcome' değişkeni analizi

X_train[['poutcome','deposit']].groupby('poutcome',as_index=False).mean().sort_values('deposit',ascending=False)

Daha önce para yatıran kişilerin tekrar para yatırdığını görebiliriz.
success için 2, other için 1 ve failure ve unknown için 0 atarız.

In [ ]:
poutcomemapping={'success':2,'other':1,'failure':0,'unknown':0}

for df in combine:
    df['poutcome']=df['poutcome'].map(poutcomemapping)

Şimdi sayısal değişken analizine bakalım

In [ ]:
#'age' değişkeni analizi
sns.distplot(X_train['age'])

In [ ]:
#skewness and kurtosis değerlerini hesaplayalım
from scipy.stats import kurtosis
from scipy.stats import skew

K=kurtosis(X_train['age'])
s=skew(X_train['age'])
print('k:',K)
print('s:',s)

K ve s değerlerinden verinin orta derecede çarpık olduğunu söylüyoruz. Çarpıklığı azaltmak için log trans uygulayalım.

In [ ]:
for df in combine:
    df['age_trns']=df['age'].apply(np.log)
    
sns.distplot(X_train['age_trns'])

In [ ]:
K=kurtosis(X_train['age_trns'])
s=skew(X_train['age_trns'])
print('k:',K)
print('s:',s)

In [ ]:
plt.figure()
x=df[df['deposit']==1]['age_trns']
y=df[df['deposit']==0]['age_trns']
plt.hist(x,bins=20,alpha=0.5,label='deposited')
plt.hist(y,bins=20,alpha=0.5,label='did not deposit')
plt.legend()
plt.show()

Yukarıdaki grafikten aşağıdaki sonuçları çıkarabiliriz.
1) Her iki geçmiş de neredeyse birbiriyle örtüşüyor
2) Gençlerin ve yaşlıların depozito yatırma olasılığının daha yüksek olduğunu görüyoruz.
Bunu kutuları oluşturarak ve analiz ederek doğrulayabiliriz.

In [ ]:
for df in combine:
    df['age_bands']=pd.cut(df['age_trns'],bins=5,precision=1)
X_train[['age_bands','deposit']].groupby('age_bands',as_index=False).mean().sort_values('deposit',ascending=False)

 Kişilere 1,2,3-1 yaş grupları ve 0, 4-2 yaş grupları atıyoruz
birleştirmede df için:
        df.loc[df['age_trns']>=4.2,'age_trns']=2
        df.loc[(df['age_trns']>=2.9) & (df['age_trns']<3.2),'age_trns']=2
        df.loc[(df['age_trns']>=3.2) & (df['age_trns']<4.2),'age_trns']=1
X_train.head()

In [ ]:
#variablebalance değişkeni analizi
plt.figure(figsize=(12,8))
sns.distplot(X_train['balance'])
K_bal=kurtosis(X_train['balance'])
s_bal=skew(X_train['balance'])
print('k:',K_bal)
print('s:',s_bal)


In [ ]:
#Bu değişkenin k ve s değerlerinin çok yüksek olduğunu görüyoruz. Log trans uygulayalım ve görelim.

for df in combine:
    df['bal_trns']=df['balance'].apply(np.cbrt)
    
sns.distplot(X_train['bal_trns'])

K_trns_bal=kurtosis(X_train['bal_trns'])
s_trns_bal=skew(X_train['bal_trns'])
print('k:',K_trns_bal)
print('s:',s_trns_bal)

In [ ]:
#S ve k değerlerimizin önemli ölçüde azaldığını görüyoruz.


In [ ]:
x=df[df['deposit']==1]['bal_trns']
y=df[df['deposit']==0]['bal_trns']
plt.figure(figsize=(12,8))
plt.hist(x,bins=10,alpha=0.5,label='deposited')
plt.hist(y,bins=10,alpha=0.5,label='did not deposit')
plt.legend()

Grafik herhangi bir sonuç çıkarmak için pek uygun gözükmüyor.
Gruplara ayıralım.

In [ ]:
X_train['bal_bands']=pd.cut(X_train['bal_trns'],bins=10)
X_train[['bal_bands','deposit']].groupby('bal_bands',as_index=False).mean().sort_values('bal_bands',ascending=False)

Kişinin banka bakiyesi ne kadar yüksek olursa, para yatırma olasılığının daha yüksek olduğunu görüyoruz.

In [ ]:
#'day' değişkeni analizi
sns.distplot(X_train['day'])

In [ ]:
x=df[df['deposit']==1]['day']
y=df[df['deposit']==0]['day']
plt.figure(figsize=(12,8))
plt.hist(x,bins=31,alpha=0.5,label='deposited')
plt.hist(y,bins=31,alpha=0.5,label='did not deposit')
plt.legend()

Verilerin neredeyse simetrik olarak dağıldığını görüyoruz ve müşterinin her ayın başında para yatırmasını beklememize rağmen, müşteriyle iletişime geçildiği günün, kişinin para yatırıp yatırmama konusunda da fazla bir etkisi olmadığını görüyoruz. insanlar genellikle her ayın başında daha fazla paraya sahip olma eğilimindedir.

In [ ]:
#'duration' değişkeni analizi

sns.distplot(X_train['duration'])
K_dur=kurtosis(X_train['duration'])
s_dur=skew(X_train['duration'])
print('k:',K_dur)
print('s:',s_dur)

In [ ]:
#Verilerin oldukça çarpık olduğunu görüyoruz. Cbrt dönüşümü uygulayalım.
for df in combine:
    df['dur_trns']=df['duration'].apply(np.cbrt)
    
sns.distplot(X_train['dur_trns'])

K_trns_dur=kurtosis(X_train['dur_trns'])
s_trns_dur=skew(X_train['dur_trns'])
print('k:',K_trns_dur)
print('s:',s_trns_dur)


In [ ]:
#Müşterinin bir arama yapmadan önce bir deopsit yapıp yapmayacağını tahmin etmemiz gerektiğinden değişken duartion'u dahil etmeyeceğiz.

In [ ]:
#'campaign' değişkeni analizi
sns.distplot(X_train['campaign'])
K_cam=kurtosis(X_train['campaign'])
s_cam=skew(X_train['campaign'])
print('k:',K_cam)
print('s:',s_cam)

In [ ]:
for df in combine:
    df['cam_trns']=df['campaign'].apply(np.log)
    
sns.distplot(X_train['cam_trns'])

K_trns_cam=kurtosis(X_train['cam_trns'])
s_trns_cam=skew(X_train['cam_trns'])
print('k:',K_trns_cam)
print('s:',s_trns_cam)

In [ ]:
x=df[df['deposit']==1]['cam_trns']
y=df[df['deposit']==0]['cam_trns']
plt.figure(figsize=(12,8))
plt.hist(x,bins=5,alpha=0.5,label='deposited')
plt.hist(y,bins=5,alpha=0.5,label='did not deposit')
plt.legend()

In [ ]:
X_train['cam_bands']=pd.cut(X_train['cam_trns'],bins=10)
X_train[['cam_bands','deposit']].groupby('cam_bands',as_index=False).mean().sort_values('deposit',ascending=False)

In [ ]:
#'pdays' değişken analizi
sns.distplot(X_train['pdays'])
K_pd=kurtosis(X_train['pdays'])
s_pd=skew(X_train['pdays'])
print('k:',K_pd)
print('s:',s_pd)

In [ ]:
for df in combine:
    df['pd_trns']=df['pdays'].apply(np.cbrt)
    
sns.distplot(X_train['pd_trns'])

K_trns_pd=kurtosis(X_train['pd_trns'])
s_trns_pd=skew(X_train['pd_trns'])
print('k:',K_trns_pd)
print('s:',s_trns_pd)

In [ ]:
x=df[df['deposit']==1]['pd_trns']
y=df[df['deposit']==0]['pd_trns']
plt.figure(figsize=(12,8))
plt.hist(x,bins=10,alpha=0.5,label='deposited')
plt.hist(y,bins=10,alpha=0.5,label='did not deposit')
plt.legend()

In [ ]:
X_train['p_bands']=pd.cut(X_train['pd_trns'],bins=10)
X_train[['p_bands','deposit']].groupby('p_bands',as_index=False).mean().sort_values('deposit',ascending=False)

In [ ]:
#Herhangi bir eğilim göremediğimiz için bu değişkeni şimdilik bırakıyoruz ve daha sonra onu çok değişkenli analizde kullanıp kullanamayacağımızı görüyoruz.

In [ ]:
#'previous' değişken analizi
sns.distplot(X_train['previous'])
K_pre=kurtosis(X_train['previous'])
s_pre=skew(X_train['previous'])
print('k:',K_pre)
print('s:',s_pre)




In [ ]:
for df in combine:
    df['pre_trns']=df['previous'].apply(np.cbrt)
    
sns.distplot(X_train['pre_trns'])

K_trns_pre=kurtosis(X_train['pre_trns'])
s_trns_pre=skew(X_train['pre_trns'])
print('k:',K_trns_pre)
print('s:',s_trns_pre)

In [ ]:
X_train['pre_bands']=pd.cut(X_train['pre_trns'],bins=10)
X_train[['pre_bands','deposit']].groupby('pre_bands',as_index=False).mean().sort_values('deposit',ascending=False)

In [ ]:
#önemli bir eğilim görülmedi, bu nedenle şimdilik bu parametreyi bırakalım

In [ ]:
X_train.columns

In [ ]:
X_train.head()

In [ ]:
X_final_train=X_train[['job','loan','month','poutcome','age_trns','bal_trns','cam_trns','dur_trns']]
X_final_test=X_test[['job','loan','month','poutcome','age_trns','bal_trns','cam_trns','dur_trns']]

In [ ]:
X_train.columns

#Karar ağacını kullanarak modelleyelim

In [ ]:
from sklearn.tree import  DecisionTreeClassifier

dt=DecisionTreeClassifier(random_state=101)
dt.fit(X_final_train,y_train)
predict=dt.predict(X_final_test)
accuracy_test=round(dt.score(X_final_test,y_test)*100,2)
accuracy_train=round(dt.score(X_final_train,y_train)*100,2)

print('train accuracy of decision tree classifier',accuracy_train)
print('test accuracy of decision tree classifier',accuracy_test)
  

In [ ]:
k_plot=[]
t_plot=[]

for k in range(1,10,1):
     dt=DecisionTreeClassifier(max_depth=k,random_state=101)
     dt.fit(X_final_train,y_train)
     predict=dt.predict(X_final_test)
     accuracy_test=round(dt.score(X_final_test,y_test)*100,2)
     accuracy_train=round(dt.score(X_final_train,y_train)*100,2)
     print(k)
     #print('train accuracy of decision tree classifier',accuracy_train)
     #print('test accuracy of decision tree classifier',accuracy_test)
     k_plot.append(accuracy_test)
     t_plot.append(accuracy_train)

In [ ]:
fig,axes=plt.subplots(1,1,figsize=(12,8))

axes.set_xticks(range(1,10,1))
k=range(1,10,1)
plt.plot(k,k_plot)
plt.plot(k,t_plot,'r')

Doğruluğun max_depth = 5'e kadar arttığını ve bunun ötesinde doğruluktaki artışın fazla olmadığını görüyoruz. O halde derinliği 5 olarak görelim

In [ ]:
dt=DecisionTreeClassifier(max_depth=5,random_state=101)
dt.fit(X_final_train,y_train)
predict=dt.predict(X_final_test)
accuracy_test=round(dt.score(X_final_test,y_test)*100,2)
accuracy_train=round(dt.score(X_final_train,y_train)*100,2)

print('train accuracy of decision tree classifier',accuracy_train)
print('test accuracy of decision tree classifier',accuracy_test)